<a href="https://colab.research.google.com/github/stvngo/Algoverse-AI-Model-Probing/blob/main/Output_Replication_Qwen_3_0_6B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and load the model
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# download the file using wget
!wget https://huggingface.co/datasets/codelion/Qwen3-0.6B-pts/raw/main/pivotal_tokens.jsonl -O pivotal_tokens.jsonl


--2025-07-31 20:35:13--  https://huggingface.co/datasets/codelion/Qwen3-0.6B-pts/raw/main/pivotal_tokens.jsonl
Resolving huggingface.co (huggingface.co)... 3.170.185.25, 3.170.185.35, 3.170.185.14, ...
Connecting to huggingface.co (huggingface.co)|3.170.185.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1466895 (1.4M) [text/plain]
Saving to: ‘pivotal_tokens.jsonl’

pivotal_tokens.json 100%[===================>]   1.40M  --.-KB/s    in 0.09s   

2025-07-31 20:35:13 (16.4 MB/s) - ‘pivotal_tokens.jsonl’ saved [1466895/1466895]



### Lightly preprocessing the data first
- Involves just loading the PTS dataset, removing duplicates, keeping only the needed fields, and filtering down to 10 unique queries for fast generation testing


In [ ]:
import json
import pandas as pd

# Load the JSONL file into a DataFrame
data_path = "pivotal_tokens.jsonl"
df = pd.read_json(data_path, lines=True)

# Drop columns we don't need (optional)
keep_cols = ["query", "pivot_context", "pivot_token", "dataset_item_id"]
df = df[keep_cols]

# Drop duplicates across all fields
df = df.drop_duplicates()

# Optional: Keep only unique queries (to avoid repeated generations)
df = df.drop_duplicates(subset="query")

# Sample 10 rows for fast debugging
df_sampled = df.sample(n=10, random_state=42)

# Save to new file for clean generation
df_sampled.to_json("preprocessed_queries.jsonl", orient="records", lines=True)

print("✅ Saved cleaned dataset to preprocessed_queries.jsonl")


✅ Saved cleaned dataset to preprocessed_queries.jsonl


In [ ]:
import torch
import json
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# Load model and tokenizer
model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", trust_remote_code=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the preprocessed pivotal tokens dataset
dataset_path = "preprocessed_queries.jsonl"
with open(dataset_path, "r") as f:
    examples = [json.loads(line) for line in f]

# Generation config
gen_config = GenerationConfig(
    temperature=0.0,
    top_p=1.0,
    top_k=0,
    do_sample=False,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,

)

# Process and generate
outputs = []
for example in tqdm(examples[:10]):  # Remove [:10] to run on full dataset
    query = example["query"]
    inputs = tokenizer(query, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(**inputs, generation_config=gen_config)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    outputs.append({
        "query": query,
        "generated": output_text,
        "expected": example.get("pivot_context", "")
    })

# Save to file
with open("generated_outputs.jsonl1", "w") as f:
    for entry in outputs:
        f.write(json.dumps(entry) + "\n")

print("✅ Done generating outputs. Saved to generated_outputs.jsonl")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
  0%|          | 0/10 [00:00<?, ?it/s]`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True, 'top_p': 0.95, 'bos_token_id': 151643, 'eos_token_id': [151645, 151643]}. If this is not desired, please set these values explicitly.
The following generation flags are not valid and may be ign

✅ Done generating outputs. Saved to generated_outputs.jsonl


In [ ]:
from google.colab import files
files.download("generated_outputs.jsonl1")

SyntaxError: invalid syntax (ipython-input-355024088.py, line 1)

In [ ]:
import os
os.path.abspath("generated_outputs.jsonl")

'/content/generated_outputs.jsonl'

In [ ]:
from google.colab import files
files.download("generated_outputs.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=89031d47b97b0633aaca060bceb6966917c0a104a4f580fccd135c6587c48f46
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
from fpdf import FPDF
import json

# Create PDF object
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Read the JSONL file
with open("generated_outputs.jsonl", "r") as f:
    for line in f:
        item = json.loads(line)
        query = item.get("query", "No query")
        generated = item.get("generated", "No generation")

        pdf.multi_cell(0, 10, f"Query: {query}\nGenerated: {generated}\n\n")

# Save to PDF
pdf.output("generated_outputs.pdf")
print("Saved to generated_outputs.pdf")


Saved to generated_outputs.pdf


In [ ]:
files.download("generated_outputs.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# PTS Output Reproduction - Debugging Setup
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import random
import numpy as np
from tqdm import tqdm

def set_all_seeds(seed=42):
    """Set all random seeds for reproducibility"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_data_and_model():
    """Load the dataset and model with exact specifications"""
    # Load data
    pts_data = []
    with open("pivotal_tokens.jsonl", "r") as f:
        for line in f:
            if line.strip():
                pts_data.append(json.loads(line))

    print(f"Loaded {len(pts_data)} entries")

    # Check exact model version from dataset
    model_id = pts_data[0]["model_id"]
    print(f"Dataset model_id: {model_id}")

    # Load model with exact same settings
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16  # Try different dtypes if needed
    )
    model.eval()

    return pts_data, model, tokenizer

def analyze_pivot_context_generation(example, model, tokenizer):
    """
    Try to understand how the pivot_context was generated
    """
    query = example["query"]
    pivot_context = example["pivot_context"]

    print(f"\nAnalyzing example:")
    print(f"Query: {query}")
    print(f"Expected pivot_context length: {len(pivot_context)} chars")
    print(f"Pivot context ends with: '...{pivot_context[-100:]}'")

    # Tokenize both to see the difference
    query_tokens = tokenizer(query, return_tensors="pt")
    context_tokens = tokenizer(pivot_context, return_tensors="pt")

    query_len = query_tokens["input_ids"].shape[1]
    context_len = context_tokens["input_ids"].shape[1]
    generated_len = context_len - query_len

    print(f"Query tokens: {query_len}")
    print(f"Full context tokens: {context_len}")
    print(f"Generated tokens: {generated_len}")

    # Extract just the generated part
    if generated_len > 0:
        generated_token_ids = context_tokens["input_ids"][0, query_len:]
        generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
        print(f"Generated part: '{generated_text}'")
        print(f"Generated token IDs: {generated_token_ids.tolist()}")

    return generated_len

def try_different_generation_strategies(query, expected_context, model, tokenizer):
    """
    Try different generation parameters to match the expected output
    """
    inputs = tokenizer(query, return_tensors="pt").to(model.device)

    # Calculate how many tokens we need to generate
    query_len = inputs["input_ids"].shape[1]
    expected_tokens = tokenizer(expected_context, return_tensors="pt")
    target_len = expected_tokens["input_ids"].shape[1] - query_len

    print(f"\nTrying to generate {target_len} tokens...")

    strategies = [
        # Strategy 1: Deterministic (greedy)
        {
            "name": "Greedy (deterministic)",
            "params": {
                "do_sample": False,
                "max_new_tokens": target_len,
                "pad_token_id": tokenizer.eos_token_id
            }
        },
        # Strategy 2: Temperature sampling
        {
            "name": "Temperature sampling (0.1)",
            "params": {
                "do_sample": True,
                "temperature": 0.1,
                "max_new_tokens": target_len,
                "pad_token_id": tokenizer.eos_token_id
            }
        },
        # Strategy 3: Original parameters (but corrected)
        {
            "name": "Original params (corrected)",
            "params": {
                "do_sample": True,
                "temperature": 0.6,
                "top_p": 0.95,
                "top_k": 20,
                "max_new_tokens": target_len,
                "pad_token_id": tokenizer.eos_token_id
            }
        },
        # Strategy 4: Very low temperature
        {
            "name": "Very low temperature (0.01)",
            "params": {
                "do_sample": True,
                "temperature": 0.01,
                "max_new_tokens": target_len,
                "pad_token_id": tokenizer.eos_token_id
            }
        }
    ]

    for strategy in strategies:
        print(f"\n--- {strategy['name']} ---")
        set_all_seeds(42)  # Reset seed before each attempt

        try:
            with torch.no_grad():
                output_ids = model.generate(
                    inputs["input_ids"],
                    **strategy["params"]
                )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            print(f"Generated: {generated_text}")

            # Check if it matches
            if expected_context.strip() in generated_text.strip():
                print("✅ MATCH FOUND!")
                return True
            else:
                # Show the difference more clearly
                generated_part = generated_text[len(query):].strip()
                expected_part = expected_context[len(query):].strip()
                print(f"Expected part: '{expected_part[:100]}...'")
                print(f"Generated part: '{generated_part[:100]}...'")

        except Exception as e:
            print(f"Error with {strategy['name']}: {e}")

    return False

def check_model_configuration(model, tokenizer):
    """Check model configuration details"""
    print("=== Model Configuration ===")
    print(f"Model config: {model.config}")
    print(f"Tokenizer vocab size: {len(tokenizer)}")
    print(f"Model vocab size: {model.config.vocab_size}")
    print(f"Model dtype: {model.dtype}")
    print(f"Device: {model.device}")

    # Check if there are any special tokens that might affect generation
    print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})")

def main():
    print("=== PTS Reproduction Debugging ===")

    # Set seeds first
    set_all_seeds(42)

    # Load everything
    pts_data, model, tokenizer = load_data_and_model()

    # Check model configuration
    check_model_configuration(model, tokenizer)

    # Analyze a few examples
    print("\n=== Analyzing Examples ===")
    for i in range(min(3, len(pts_data))):
        example = pts_data[i]
        if not example.get("query") or not example.get("pivot_context"):
            continue

        print(f"\n{'='*50}")
        print(f"EXAMPLE {i+1}")
        print(f"{'='*50}")

        # Analyze the structure
        generated_len = analyze_pivot_context_generation(example, model, tokenizer)

        if generated_len > 0:
            # Try to reproduce
            success = try_different_generation_strategies(
                example["query"],
                example["pivot_context"],
                model,
                tokenizer
            )

            if success:
                print("🎉 Successfully reproduced this example!")
                break
        else:
            print("⚠️ This example has no generated content (pivot_context = query)")

if __name__ == "__main__":
    main()

=== PTS Reproduction Debugging ===
Loaded 971 entries
Dataset model_id: Qwen/Qwen3-0.6B


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Model Configuration ===
Model config: Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embedding

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated: A tomato plant has 100 tomatoes. Jane picks 1/4 of that number for use in their house. After a week, she goes back and picks 20 more tomatoes, and the following week picks twice that number. What's the total number of fruits remaining on the tomato plant? Let's solve this problem step by step.

First, let's understand the problem. The tomato plant starts with 100 tomatoes. Jane picks 1/4 of that number. Then, after a week, she picks 20 more tomatoes. Then, the following week, she picks twice that number. We need to find the total number of fruits remaining on the tomato plant.

Let's break it down step by step.

Step 1:
Expected part: 'Let me think this through.

Okay, let's see. There are 100 tomatoes on the plant. Jane picks 1/4 of ...'
Generated part: 'Let's solve this problem step by step.

First, let's understand the problem. The tomato plant starts...'

EXAMPLE 2

Analyzing example:
Query: A tomato plant has 100 tomatoes. Jane picks 1/4 of that number for use in their 

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generated: A tomato plant has 100 tomatoes. Jane picks 1/4 of that number for use in their house. After a week, she goes back and picks 20 more tomatoes, and the following week picks twice that number. What's the total number of fruits remaining on the tomato plant? Let's solve this problem step by step.

First, let's understand the problem. The tomato plant starts with 100 tomatoes. Jane picks 1/4 of that number. Then, after a week, she picks 20 more tomatoes. Then, the following week, she picks twice that number. We need to find the total number of fruits remaining on the tomato plant.

Let's break it down step by step.

Step 1: Initial
Expected part: 'Let me think this through.

Okay, let's see. There are 100 tomatoes on the plant. Jane picks 1/4 of ...'
Generated part: 'Let's solve this problem step by step.

First, let's understand the problem. The tomato plant starts...'

EXAMPLE 3

Analyzing example:
Query: A tomato plant has 100 tomatoes. Jane picks 1/4 of that number for use i